# Aprendizem Profunda - Simple RNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [24]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [25]:
df = pd.read_csv("data/big_dataset.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [26]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [27]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [28]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 24276, Unique texts: 24158
Overlap between train and test after deduplication: 0


### Tokenizer

In [29]:
# Tokenize the data
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [30]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=200)  

### Early Stopping

In [31]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=5,            
    restore_best_weights=True,           
)

### Reduce_lr

In [32]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## Simple RNN

In [35]:
model_simple_rnn = Sequential([
    Embedding(input_dim=30000, output_dim=128, input_length=200), 
    SimpleRNN(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),  
    SimpleRNN(32, dropout=0.3, recurrent_dropout=0.3),  
    BatchNormalization(),
    Dense(32, activation="relu", kernel_regularizer=l2(0.005)),  
    Dropout(0.5),
    Dense(1, activation="sigmoid", kernel_regularizer=l2(0.005))
])

optimizer = Adam(learning_rate=0.001)
model_simple_rnn.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

model_simple_rnn.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)

Epoch 1/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 51s 96ms/step - accuracy: 0.5045 - loss: 0.9276 - val_accuracy: 0.4997 - val_loss: 0.7666 - learning_rate: 0.0010
Epoch 2/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 49s 102ms/step - accuracy: 0.5043 - loss: 0.7576 - val_accuracy: 0.4912 - val_loss: 0.7269 - learning_rate: 0.0010
Epoch 3/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 52s 108ms/step - accuracy: 0.5026 - loss: 0.7220 - val_accuracy: 0.5057 - val_loss: 0.7081 - learning_rate: 0.0010
Epoch 4/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 54s 111ms/step - accuracy: 0.5058 - loss: 0.7055 - val_accuracy: 0.5062 - val_loss: 0.6997 - learning_rate: 0.0010
Epoch 5/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 48s 99ms/step - accuracy: 0.5058 - loss: 0.6986 - val_accuracy: 0.5057 - val_loss: 0.6958 - learning_rate: 0.0010
Epoch 6/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 49s 102ms/step - accuracy: 0.5077 - loss: 0.6955 - val_accuracy: 0.5057 - val_loss: 0.6946 - learning_rate: 0.0010
Epoch 7/50
484/484 ━━━━━━━━━━━━━━━━━━━━ 85s 109ms/step - accuracy: 0.500

# Correr Modelos

**Test Set**

In [ ]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_simple_rnn, X_test, test_ids, "simple_rnn")



151/151 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


**Avaliação**

In [ ]:
predict_and_save(model_simple_rnn, X_professor, data_set_professor["ID"], "simple_rnn_professor")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step


## Classification Reports

In [ ]:
print(classification_report(y_test, model_simple_rnn.predict(X_test) > 0.5))

151/151 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
              precision    recall  f1-score   support

         0.0       0.92      0.81      0.86      2443
         1.0       0.83      0.93      0.87      2389

    accuracy                           0.87      4832
   macro avg       0.87      0.87      0.87      4832
weighted avg       0.87      0.87      0.87      4832

